# Section 5-6: Use the sampler/estimator primitive

## TASK 5.1/6.1: Set sampler/estimator primitive options

1. Which snippet correctly transpiles a parameterized circuit for a chosen backend and produces an ISA circuit before using Estimator V2?

a. 
```python
from qiskit.transpiler import generate_preset_pass_manager
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circ = pm.run(circ)
```
b. 
```python
isa_circ = circ  # QPUs accept any gates; transpilation is optional
```
c. 
```python
from qiskit import transpile
isa_circ = transpile(circ)  # omit backend
```
d. 
```python
pm = generate_preset_pass_manager(backend=None)
isa_circ = pm.run(circ.decompose())
```


:::{dropdown}answer 
The answer is a. 

You must generate a preset pass manager with a concrete `backend` and run it to obtain an ISA circuit; this matches the example that uses `generate_preset_pass_manager(optimization_level=1, backend=backend)` followed by `pm.run(circuit)`. This step ensures the circuit uses only instructions supported by the target device prior to invoking primitives.
:::


2. After transpiling a circuit for Estimator, how do you correctly align the observable to the transpiled circuit layout?

a. 
```python
isa_obs = observable.apply_layout(isa_circ.layout)
```
b. 
```python
isa_obs = observable.transpile_like(isa_circ)
```
c. 
```python
isa_obs = observable.bind_parameters(isa_circ.parameters)
```
d. 
```python
isa_obs = observable.measure_all(inplace=True)
```


:::{dropdown}answer 
The answer is a. 

The documented workflow applies the transpiled circuit’s layout to the observable via `observable.apply_layout(isa_circuit.layout)` so that qubit indices match the ISA circuit fed to Estimator.

```python
from qiskit.transpiler import generate_preset_pass_manager
 
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circuit = pm.run(circuit)
isa_observable = observable.apply_layout(isa_circuit.layout)
print(f">>> Circuit ops (ISA): {isa_circuit.count_ops()}")

# >>> Circuit ops (ISA): OrderedDict([('rz', 2734), ('sx', 1185), ('ecr', 576), ('x', 284)])
```
:::


3. Which call correctly submits work to Estimator V2 using the Primitive Unified Bloc (PUB) tuple?

a. 
```python
job = estimator.run([(isa_circ, isa_obs, param_values)])
```
b. 
```python
job = estimator.run([(isa_obs, isa_circ, param_values)])
```
c. 
```python
job = estimator.run([(isa_circ, param_values)])
```
d. 
```python
job = estimator.run((isa_circ, isa_obs, param_values))
```


:::{dropdown}answer 
The answer is a. 

The Estimator expects a sequence of PUB tuples in the form `(circuit, observable, parameter_values)`; the example shows `estimator.run([(isa_circuit, isa_observable, param_values)])`.
:::


4. For Sampler V2, which preparation is correct before transpilation and submission?

a. 
```python
circ.measure_all()
pm = generate_preset_pass_manager(optimization_level=1, backend=backend)
isa_circ = pm.run(circ)
job = sampler.run([(isa_circ, param_values)])
```
b. 
```python
pm = generate_preset_pass_manager(optimization_level=3, backend=backend)
isa_circ = pm.run(circ)  # no measurements needed
job = sampler.run([(isa_circ,)])
```
c. 
```python
job = sampler.run([(circ,)])
```
d. 
```python
isa_circ = circ.remove_final_measurements(inplace=True)
job = sampler.run([(isa_circ,)])
```


:::{dropdown}answer 
The answer is a. 

The sampler example measures the circuit (`measure_all()`), transpiles to ISA with a preset pass manager for the chosen backend, and submits `[(isa_circuit, param_values)]` as the PUB.
:::


5. Which initialization correctly selects the execution mode consistent with the examples?

a. 
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator(mode=backend)
```
b. 
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator(mode="session")  # string literal
```
c. 
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator()  # mode is mandatory
```
d. 
```python
from qiskit_ibm_runtime import EstimatorV2 as Estimator
estimator = Estimator(mode="batch").run(backend)
```


:::{dropdown}answer 
The answer is a. 

When you initialize the Estimator, use the `mode` parameter to specify the mode you want it to run in. Possible values are `batch`, `session`, or `backend` objects for batch, session, and job execution mode, respectively.
:::


6. How can you request a backend that supports fractional gates per the guide?

a. 
```python
service = QiskitRuntimeService()
backend = QiskitRuntimeService(use_fractional_gates=True)
```
b. 
```python
service = QiskitRuntimeService()
backend = service.get_backend("fractional-gates")
```
c. 
```python
service = QiskitRuntimeService()
backend = service.least_busy(fractional=True)
```
d. 
```python
service = QiskitRuntimeService()
backend = service.least_busy(use_fractional_gates=True)
```


:::{dropdown}answer 
The answer is d. 

To use the newly supported [fractional gates](https://quantum.cloud.ibm.com/docs/guides/fractional-gates), set `use_fractional_gates=True` when requesting a backend from a `QiskitRuntimeService` instance. 

### What are fractional gates

Fractional gates are native hardware operations (e.g., **R\_X(θ)**, **R\_ZZ(θ)**) that allow arbitrary-angle rotations to be executed directly on supported IBM Quantum devices, without decomposing into multiple basis gates.

---

### When to use

* Circuits with many arbitrary rotations (e.g., VQE, QAOA, quantum simulation).
* To **reduce circuit depth**, execution time, and cumulative error.
* When the backend supports fractional gates and constraints (like angle ranges) are met.

---

### When not to use (and why)

* **Unsupported cases**: dynamic circuits, certain error-mitigation techniques (Pauli twirling, PEC, ZNE with PEA).
* **Restricted angles**: R\_ZZ(θ) only supports 0 < θ ≤ π/2.
* **Runtime parameters** may cause errors if angles are out of range.
* For simple circuits where decomposition overhead is negligible.
* It’s still an **experimental feature**, so stability and compatibility can change.
:::


7. Which code path best reflects a minimal, correct optimization/transpilation workflow before running primitives?

a. Create circuit → `generate_preset_pass_manager(optimization_level=1, backend)` → `pm.run(circuit)` → align observable with `apply_layout` → `Estimator.run([...])`<br>
b. Create circuit → call `Estimator.run(circuit)` and let the service infer gates automatically<br>
c. Create circuit → decompose to basis gates locally without a backend → `Estimator.run([...])`<br>
d. Create circuit → add barriers to prevent changes → `Estimator.run([...])`<br>


:::{dropdown}answer 
The answer is a. 

The documented steps show generating a preset pass manager with a specified backend, running it to obtain an ISA circuit, aligning the observable, and then submitting PUBs to the primitive.
:::


8. Which code correctly enables dynamical decoupling (DD) in Sampler V2 and selects the XpXm sequence?

a. 
```
sampler.options.dynamical_decoupling.enable = True
sampler.options.dynamical_decoupling.sequence_type = "XpXm"
```

b. 
```
sampler.options.resilience.zne_mitigation = True
sampler.options.resilience.zne.extrapolator = "exponential"
```

c. 
```
sampler.options.twirling.enable_gates = True
sampler.options.twirling.sequence_type = "XpXm"
```

d. 
```
sampler.set_options(dynamical_decoupling=True, sequence_type="XpXm")
```


:::{dropdown}answer 
The answer is a. 

Sampler V2 exposes DD via `sampler.options.dynamical_decoupling`. Set `enable=True` and choose a valid `sequence_type` such as `"XpXm"`. Error mitigation like ZNE belongs to the Estimator, not Sampler. 

### Dynamic Decoupling
Dynamical decoupling works by inserting pulse sequences on idling qubits to approximately cancel out the effect of these errors. Each inserted pulse sequence amounts to an identity operation, but the physical presence of the pulses has the effect of suppressing errors. 
:::


9. Select all valid values for `sampler.options.dynamical_decoupling.sequence_type`. (Multiple answers)

a. "XX"<br>
b. "XpXm"<br>
c. "XY4"<br>
d. "XZXZ"<br>


:::{dropdown}answer 
The answer is a, b, c. 

Valid DD sequence types are `"XX"`, `"XpXm"`, and `"XY4"`. `"XZXZ"` is not a supported built‑in option. 
:::


10. When is enabling dynamical decoupling most likely to help for Sampler runs on hardware?

a. Only when running on a simulator with noise disabled<br>
b. When all qubits are busy almost all the time (densely packed schedules)<br>
c. When circuits contain idle gaps on some qubits where no operations occur<br>
d. Only when measuring expectation values with Estimator<br>


:::{dropdown}answer 
The answer is c. 

DD is primarily beneficial when there are idle periods; pulses inserted during idle intervals can suppress coherent errors. With densely packed schedules, DD can provide little benefit or even hurt due to pulse imperfections. 
:::


11. Which error‑related options are supported directly by Sampler V2?

a. Pauli twirling (suppression) via `sampler.options.twirling.*`<br>
b. TREX measurement mitigation via `sampler.options.resilience.measure_mitigation = True`<br>
c. Zero‑noise extrapolation via `sampler.options.resilience.zne_mitigation = True`<br>
d. Probabilistic error cancellation via `sampler.options.resilience.pec_mitigation = True`<br>


:::{dropdown}answer 
The answer is a. 

Sampler supports suppression methods like Pauli twirling and dynamical decoupling. Mitigation techniques such as TREX, ZNE, and PEC are available in Estimator; Sampler does not support them directly. 
:::


12. Which snippet correctly configures Pauli twirling in Sampler V2 with 32 randomizations and 100 shots per randomization?

a. 
```
sampler.options.twirling.enable_gates = True
sampler.options.twirling.num_randomizations = 32
sampler.options.twirling.shots_per_randomization = 100
```

b. 
```
sampler.options.twirling.enable = True
sampler.options.twirling.count = 32
sampler.options.twirling.shots = 100
```

c. 
```
sampler.options.resilience.twirling = True
sampler.options.resilience.num_randomizations = 32
sampler.options.resilience.shots_per_randomization = 100
```

d. 
```
sampler.enable_twirling(32, 100)
```

:::{dropdown}answer 
The answer is a. 

Use the `twirling` options on Sampler: set `enable_gates=True`, then configure `num_randomizations` and `shots_per_randomization`.
- `num_randomizations`: The number of circuit instances to draw from the ensemble of twirled circuits.
- `shots_per_randomization`: The number of shots to sample from each circuit instance.
Here’s a concise English summary of the **Concept** and **Motivation** sections of *Pauli twirling*, along with an explanation of the difference between **coherent** and **incoherent noise**:

---

### Pauli twirling

Pauli twirling is a technique where random Pauli operators (I, X, Y, Z) are inserted before and after certain gates in a quantum circuit. This process effectively transforms arbitrary noise into a *Pauli channel*, a simpler and more tractable noise model.

Quantum hardware noise often contains **coherent errors**, which accumulate *quadratically* with the number of operations, making them especially harmful. By applying Pauli twirling, these coherent errors are randomized and converted into **incoherent (Pauli) errors**, which accumulate *linearly* with circuit depth. This makes the errors easier to analyze and mitigate, and it allows error mitigation techniques that assume Pauli noise to be more effective.

---

### Coherent vs. Incoherent errors

* **Coherent errors**:

  * Systematic and unitary in nature (e.g., consistent over-rotation or phase error in a gate).
  * Error amplitudes interfere constructively over repeated operations, causing error growth to scale quadratically.
* **Incoherent errors**:

  * Probabilistic and stochastic (e.g., random bit flips, depolarizing noise).
  * Errors accumulate additively, leading to linear error growth with circuit depth.

:::


13. How do you set a repetition delay of 0.5 ms between shots for Sampler V2?

a. sampler.options.execution.rep_delay = 0.5<br>
b. sampler.options.environment.rep_delay = 0.5<br>
c. sampler.options.execution.rep_delay = 0.0005<br>
d. sampler.set_options(rep_delay_ms=0.5)<br>


:::{dropdown}answer 
The answer is c. 

`rep_delay` is specified in seconds within the `execution` options; 0.5 ms equals 0.0005 seconds. 
:::


14. What is the default DD sequence used if you only enable `sampler.options.dynamical_decoupling.enable = True` and do not set `sequence_type`?

a. "XX"<br>
b. "XpXm"<br>
c. "XY4"<br>
d. There is no default; you must choose explicitly<br>


:::{dropdown}answer 
The answer is a. 

The default dynamical decoupling sequence is `"XX"` unless you override it. 
:::


15. Which statement about dynamical decoupling is TRUE?

a. DD is a measurement‑error mitigation method that post‑processes bitstrings<br>
b. Each inserted DD pulse sequence ideally implements an identity operation while suppressing coherent errors on idling qubits<br>
c. DD is guaranteed to improve results in all cases, even with dense schedules<br>
d. DD can only be used with Estimator, not with Sampler<br>


:::{dropdown}answer 
The answer is b. 

DD inserts carefully chosen pulse sequences during idle gaps. These sequences act as identity operations but help cancel coherent errors. It is a suppression (control) technique, not a post‑processing mitigation method. 
:::


16. In EstimatorV2, what does `options.default_precision` control and what is its default value?

a. It sets the target standard error for expectation values (used when shots aren’t given); default is 0.015625 (1/√4096).<br>
b. It sets the transpiler optimization level; default is 1.<br>
c. It fixes the number of shots for each circuit configuration; default is 4096.<br>
d. It controls backend run priority; default is “normal”.<br>


:::{dropdown}answer 
The answer is a. 

`default_precision` defines the default statistical precision target for expectation values when neither the PUB nor `run()` specifies precision; EstimatorV2 chooses shots to meet this. The documented default is 0.015625 (i.e., 1/√4096).
:::


17. Regarding `options.default_shots` in EstimatorV2, which statement is correct?

a. If set, it overrides `default_precision`, and when twirling is enabled it is split across randomizations.<br>
b. It is ignored whenever twirling is enabled.<br>
c. It only applies to simulators and is ignored on hardware.<br>
d. It sets the maximum number of shots per job but never overrides precision.<br>


:::{dropdown}answer 
The answer is a. 

When `default_shots` is provided, it takes precedence over `default_precision`. If Pauli twirling is active, the total shots are divided among the circuit randomizations according to the twirling options.
:::


18. Which mapping of `resilience_level` to behavior is correct for EstimatorV2?

a. 0: No mitigation; 1: Readout mitigation + measurement twirling; 2: Adds bias‑reducing estimator mitigation (medium cost).<br>
b. 0: Readout mitigation only; 1: Full error correction; 2: No mitigation.<br>
c. 0: Measurement twirling; 1: Probabilistic error cancellation; 2: Zero‑noise extrapolation only.<br>
d. 0: No mitigation; 1: Full QEC; 2: Hardware calibration reset only.<br>


:::{dropdown}answer 
The answer is a. 

`resilience_level` increases mitigation complexity: 
- 0 disables mitigation
- 1 applies minimal‑cost techniques like readout mitigation (and measurement twirling in V2)
- 2 adds medium‑cost estimator bias‑reduction methods
:::


19. For `TwirlingOptions.strategy`, what best describes the default `"active-accum"` behavior?

a. In each twirled layer, twirl the union of instruction qubits encountered up to that layer.<br>
b. Twirl only the instruction qubits of the current layer, ignoring earlier ones.<br>
c. Twirl all qubits in the circuit at every twirled layer.<br>
d. Twirl only the first two qubits of the circuit to minimize overhead.<br>


:::{dropdown}answer 
The answer is a. 

`"active-accum"` accumulates the set of instruction qubits seen so far and twirls that growing set at each subsequent twirled layer, which is default strategy.
:::


20. What is the default of `TwirlingOptions.enable_measure` and where does it apply?

a. `True` for Estimator, `False` for Sampler; enables twirling on measurement instructions not inside conditionals.<br>
b. `True` for both Estimator and Sampler; always twirl all measurements.<br>
c. `False` for both primitives; measurement twirling must be enabled via `resilience_level` only.<br>
d. `True` only on simulators; `False` on hardware backends.<br>


:::{dropdown}answer 
The answer is a. 

Measurement twirling is enabled by default for Estimator and disabled by default for Sampler. It applies to measurement operations provided they are not within a conditional block.
:::


21. When `options.resilience.zne.amplifier='pea'` is selected, which effect is correct?

a. The runtime always uses twirling and performs layer noise learning before executing amplified circuits.<br>
b. ZNE is disabled because PEA conflicts with twirling.<br>
c. Only single‑qubit gates are folded; two‑qubit gates are untouched.<br>
d. The extrapolator is forced to `'fallback'`.<br>


:::{dropdown}answer 
The answer is a. 

With PEA (Probabilistic Error Amplification), the runtime learns a twirled noise model per entangling layer (layer noise learning) and applies twirling automatically, then injects noise proportional to the learned model during execution.
:::


22. What are the documented defaults for `options.resilience.zne.noise_factors`?

a. `(1, 1.5, 2)` for PEA and `(1, 3, 5)` otherwise.<br>
b. `(1, 2, 4)` for PEA and `(1, 3, 9)` otherwise.<br>
c. Always `(1, 3, 5)` regardless of amplifier.<br>
d. Determined solely by `default_precision`.<br>


:::{dropdown}answer 
The answer is a. 

By default, PEA uses smaller incremental factors `(1, 1.5, 2)`, while gate‑folding–based amplification uses `(1, 3, 5)`.
:::


23. When ZNE options are enabled in EstimatorV2, which additional data fields are returned per PUB result?

a. `evs_extrapolated`/`stds_extrapolated` and `evs_noise_factors`/`stds_noise_factors`/`ensemble_stds_noise_factors`.<br>
b. Only `calibration_matrix` with no standard deviations.<br>
c. `zne_trace` strings and `fit_r2` only.<br>
d. No additional data; ZNE affects only `evs`/`stds`.<br>


:::{dropdown}answer 
The answer is a. 

With ZNE, in addition to `evs` and `stds`, the runtime returns extrapolated evaluations and per‑noise‑factor evaluations (including ensemble standard deviations) with shapes defined by the observable and parameter broadcasting.
:::


24. Which combination of ResilienceOptionsV2 booleans is consistent with the documented defaults?

a. `measure_mitigation=True`, `zne_mitigation=False`, `pec_mitigation=False`.<br>
b. `measure_mitigation=False`, `zne_mitigation=True`, `pec_mitigation=False`.<br>
c. `measure_mitigation=True`, `zne_mitigation=True`, `pec_mitigation=True`.<br>
d. `measure_mitigation=False`, `zne_mitigation=False`, `pec_mitigation=True`.<br>


:::{dropdown}answer 
The answer is a. 

Measurement error mitigation is enabled by default; ZNE and PEC are off by default unless explicitly enabled (and can be fine‑tuned via their sub‑options).
:::


25. What is `options.seed_estimator` used for in EstimatorV2?

a. To control sampling randomness (e.g., shot sampling and randomizations) via a fixed seed.<br>
b. To seed the transpiler’s layout selection.<br>
c. To fix the hardware qubit mapping across sessions.<br>
d. To seed the optimizer used in VQE.<br>


:::{dropdown}answer 
The answer is a. 

`seed_estimator` sets a seed to control stochastic parts of the estimator’s execution, such as sampling and randomized procedures, improving reproducibility across runs.
:::


## TASK 5.2/6.2: Understand the theoretical background behind the sampler/estimator primitive

1. What is the primary role of the **Sampler primitive** in Qiskit Runtime?

a. To estimate expectation values of observables from quantum circuits<br>
b. To sample bitstrings from quantum circuits and return probability distributions<br>
c. To automatically compile and optimize quantum circuits into hardware basis gates<br>
d. To implement error correction by dynamically repeating measurements<br>

:::{dropdown}answer
The answer is b.

The Sampler primitive is designed to run quantum circuits and return **quasi-probability distributions** of measurement outcomes (bitstrings). It does not calculate expectation values (that is the Estimator’s role), nor does it perform compilation or error correction itself.
:::

2. Which of the following best describes the **output** of the Sampler primitive?

a. A histogram of expectation values for given observables<br>
b. A quasi-probability distribution over bitstrings measured from the circuit<br>
c. A sequence of transpiled circuits optimized for the backend<br>
d. A set of error-mitigated observables in operator form<br>

:::{dropdown}answer
The answer is b.

The Sampler outputs a **quasi-probability distribution** that summarizes how often each bitstring is expected to occur. This distribution captures sampling statistics, while expectation values belong to Estimator results.
:::

3. From a theoretical perspective, how does the **Sampler primitive** relate to projective measurements in quantum mechanics?

a. It implements an approximation of eigenvalue estimation for arbitrary operators<br>
b. It directly samples computational basis states according to Born’s rule<br>
c. It generates a classical approximation of wavefunction amplitudes<br>
d. It collapses the state into eigenstates of Pauli operators only<br>

:::{dropdown}answer
The answer is b.

The Sampler models **Born’s rule**: after a quantum circuit is executed, measurements in the computational basis yield outcomes with probabilities proportional to the squared amplitudes of the final state vector. Sampler automates this sampling process.
:::

4. Which scenario is most appropriate for using the **Sampler primitive** instead of the Estimator?

a. When one needs to calculate expectation values of a Hamiltonian<br>
b. When one needs raw measurement outcomes or probability distributions from circuits<br>
c. When one needs to optimize circuit depth before hardware execution<br>
d. When one needs to compute gradients of parameterized circuits<br>

:::{dropdown}answer
The answer is b.

The Sampler is suited for tasks where **raw measurement statistics** (bitstring distributions) are needed, such as training a variational algorithm using measured samples. For Hamiltonian expectation values, the Estimator is the correct primitive.
:::

5. What theoretical advantage does using the **Sampler primitive** provide over manually executing circuits with `backend.run()`?

a. It provides guaranteed error correction of results<br>
b. It automatically transforms any observable into Pauli terms<br>
c. It offers a higher-level abstraction focused on probability distributions rather than low-level job control<br>
d. It bypasses the need for quantum circuit transpilation<br>

:::{dropdown}answer
The answer is c.

The Sampler simplifies the user workflow by exposing a **high-level interface** that directly provides probability distributions of bitstrings. Unlike `backend.run()`, it hides low-level details such as job management, allowing developers to focus on sampling tasks.
:::

6. In the context of the Estimator primitive, what is the main theoretical effect of **Pauli twirling** on gate errors?

a. It only applies to single-qubit gates and leaves two-qubit noise unchanged.<br>
b. It eliminates measurement (readout) errors by repeating measurements with majority voting.<br>
c. It exactly cancels hardware noise by inverting it gate-by-gate.<br>
d. It transforms arbitrary noise into a Pauli channel so that coherent errors accumulate linearly rather than quadratically.<br>

:::{dropdown}answer 
The answer is d. 

Pauli twirling (randomized compiling with Pauli gates) maps general noise to a Pauli channel. This changes error growth from quadratic (coherent) to linear (Pauli), improving expectation-value estimation by the Estimator when combined with sampling over randomized circuit instances. 
:::


7. **Twirled readout error extinction (TREX)** relies on “measurement twirling.” What key property does this introduce for the readout-error transfer matrix?

a. It makes readout errors vanish entirely without any calibration overhead.<br>
b. It leaves the transfer matrix unchanged but reduces its determinant to 1.<br>
c. It diagonalizes the transfer matrix, making it straightforward to invert for mitigation.<br>
d. It converts all Z-basis measurements to X-basis to average out noise.<br>

:::{dropdown}answer 
The answer is c. 

By randomly inserting an X gate, measuring, and classically flipping the bit, measurement twirling diagonalizes the readout-error transfer matrix. That structure enables simple inversion (after running small calibration circuits), reducing bias in expectation values returned by the Estimator.
:::


3. **Zero-noise extrapolation (ZNE)**, as used with the Estimator, is best summarized by which two-stage process?

a. Learn a quasi-probability model and sample from it to reproduce the ideal circuit exactly.<br>
b. Amplify noise (e.g., by digital gate folding) and then extrapolate expectation values to the zero-noise limit.<br>
c. Insert idle-pulse sequences to cancel coherent errors during “gaps” in the schedule.<br>
d. Calibrate a confusion matrix and invert it to de-bias measurement outcomes.<br>

:::{dropdown}answer 
The answer is b. 

ZNE runs the same logical circuit at different effective noise levels (e.g., via gate folding like replacing \(U\) with \(UU^\dagger U\)), then fits an extrapolator (linear, exponential, etc.) to estimate the zero-noise expectation value.
:::


4. In **digital gate folding** for ZNE, replacing a unitary $U$ by $UU^{\dagger}U$ corresponds to which **noise amplification factor**?

a. 3<br>
b. 2<br>
c. 1<br>
d. 0<br>

:::{dropdown}answer 
The answer is a. 

Qiskit Runtime implements noise amplification by "digital gate folding," which means that two-qubit gates are replaced with equivalent sequences of the gate and its inverse.

The sequence $UU^{\dagger}U$ increases the effective number of noisy operations by a factor (`noise_factors`) of 3 while preserving the ideal unitary action, enabling controlled noise amplification for extrapolation.

- noise_factors: The noise factors to use for noise amplification.
:::


5. **Probabilistic Error Amplification (PEA)** provides a more accurate noise amplification than simple folding. Which statement captures its core idea?

a. It learns a twirled noise model for each entangling layer and then injects single-qubit noise proportionally during amplification.<br>
b. It only repeats two-qubit gates and their inverses to scale noise without any learning.<br>
c. It yields an exactly unbiased estimator of the expectation value without overhead.<br>
d. It does not require ZNE to be enabled because it is a suppression (not mitigation) method.<br>

:::{dropdown}answer 
The answer is a. 

PEA first **learns** the twirled noise per entangling layer, then **amplifies** by probabilistically injecting single‑qubit noise consistent with that learned model before **extrapolating** to the zero‑noise limit. It is typically used alongside ZNE in Estimator workflows.
:::


6. For **probabilistic error cancellation (PEC)** applied with the Estimator, which statement about sampling overhead is correct?

a. The required shots scale with the square of the quasi‑probability negativity $\gamma$; $\gamma$ itself tends to grow exponentially with circuit depth.<br>
b. The overhead is linear in $\gamma$ and independent of circuit depth.<br>
c. When some coefficients are negative, the overhead drops to zero due to cancellation.<br>
d. PEC returns a biased estimator unless combined with measurement twirling.<br>

:::{dropdown}answer 
The answer is a. 

PEC expresses the ideal circuit as a quasi‑probability mixture of noisy circuits. The sampling overhead multiplies the shot count by roughly $\gamma^2$, where $\gamma = \sum_i |\eta_i|$ reflects the negativity in the expansion and typically grows with depth.
:::


7. In **Pauli gate twirling** as implemented for Estimator runs, how are **randomized circuit instances** and shots typically managed?

a. They change the backend’s native basis gates to Pauli-only operations.<br>
b. Randomizations only affect calibration circuits and never the main circuit instances.<br>
c. The parameters set the ZNE noise factors and the extrapolator form.<br>
d. The Estimator samples multiple randomized instances (controlled by `num_randomizations`) and takes `shots_per_randomization` shots from each.<br>

:::{dropdown}answer 
The answer is d. 

Gate twirling replaces a single circuit with an ensemble of randomized, Pauli‑wrapped instances that have the same ideal effect. The Estimator then draws a specified number of instances and shots per instance to estimate expectation values.
:::


8. For **gate twirling**, which hardware layer is **most commonly targeted** and why?

a. Single‑qubit gates, because they typically have the largest error rates.<br>
b. Measurement operations, because they dominate coherent over-rotations.<br>
c. Native two‑qubit gates, because they dominate error on current devices.<br>
d. Virtual Z gates, because they introduce significant stochastic errors.<br>

:::{dropdown}answer 
The answer is c. 

Two‑qubit operations are typically the largest contributors to gate error in today’s hardware, so twirling is often restricted to those native gates (e.g., CNOT or ECR) to reshape noise in a beneficial way.
:::


9. Conceptually, how do **error suppression** and **error mitigation** differ for Estimator‑based workflows?

a. Suppression modifies the implemented circuit/pulse schedule to reduce noise during execution (e.g., DD, gate twirling), whereas mitigation post‑processes or re‑processes experimental data to adjust expectation values (e.g., TREX, ZNE, PEC).<br>
b. Both terms mean the same thing and are interchangeable in practice.<br>
c. Mitigation changes gate calibrations in hardware; suppression extrapolates to the zero‑noise limit.<br>
d. Suppression is only about measurement, while mitigation is only about two‑qubit gates.<br>

:::{dropdown}answer 
The answer is a. 

Suppression techniques alter what is executed (added pulses, randomized wrappers) to reduce error accumulation, while mitigation techniques change how results are **estimated** (diagonalizing readout noise, extrapolating to zero noise, or canceling errors via quasi‑probabilities) to recover better expectation values.
:::
